In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_2000.p', 'rb'))

Using TensorFlow backend.


[]


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])
aspects = [str(i) for i in range(4)]

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
idxs = val_idxs
H = {}
labs = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
#     H[str(i)][np.ix_(a0, a0)] = 1
#     H[str(i)][np.ix_(a1, a1)] = 1
    
    labs[str(i)] = np.array([0] * len(idxs))
    labs[str(i)][a1] = 1
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [5]:
from NVDMModel import log_softmax, cross_ent_loss, sample_norm

In [6]:
from keras.models import load_model
# model = load_model('../store/weights/beer/Gated2CNNModelWedDec616:48:472017gatedfinal/loss.h5', 
#                    custom_objects={'contrastive_loss' : lambda a,b:b})
#model = load_model('../store/weights/beer/CountModelWedDec620:08:322017coutnfinal/loss.h5',
#                   custom_objects={'contrastive_loss' : lambda a,b:b})
model = load_model('../store/weights/beer/Gated2CNNModelSunDec1015:45:442017testingsigmoid/loss.h5',
                  custom_objects={'contrastive_loss' : lambda a,b:b, 'log_softmax' : log_softmax,
                                 'cross_ent_loss' : cross_ent_loss, 'sample_norm' : sample_norm,
                                 '<lambda>' : lambda a,b : a})

In [7]:
import keras.backend as K
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(
                model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs + [K.learning_phase()], 
                model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).outputs)

In [8]:
X_s = beer_data.X[val_idxs]
# X_tf = np.zeros((X_s.shape[0], beer_data.vocab_size))
# for i in range(len(X_s)) :
#     X_tf[i, X_s[i, :]] = 1.

# X_tf = X_tf[:, 2:]
# X_s = X_tf

In [9]:
embeds = {}
vecs = {k:[] for k in aspects}
i, bs = 0, 100
while i*bs < len(val_idxs):
    for aspect in aspects :
        result = model_c1[aspect]([X_s[i*bs:(i+1)*bs], 0])[0]
        vecs[aspect].append(result)
    i += 1

for aspect in aspects :
    embeds[aspect] = np.concatenate(vecs[aspect], axis=0)


In [11]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.919458938317
0 1 0.890791133054
0 2 0.888964736673
0 3 0.878375715957
1 0 0.895124552889
1 1 0.93424373024
1 2 0.915162573522
1 3 0.921256197243
2 0 0.903979074933
2 1 0.92566484676
2 2 0.937620831213
2 3 0.931992225086
3 0 0.886492009786
3 1 0.94115653166
3 2 0.947768928205
3 3 0.965067441015


In [16]:
#Count 
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.891117629855
0 1 0.857810746154
0 2 0.84943157754
0 3 0.83754084048
1 0 0.884981179082
1 1 0.9061766209
1 2 0.894146949952
1 3 0.893188805196
2 0 0.890444927586
2 1 0.908702550073
2 2 0.914806296358
2 3 0.908085598575
3 0 0.874131910989
3 1 0.920884309361
3 2 0.921607899325
3 3 0.933467855926


In [26]:
#NVDM
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.886122322955
0 1 0.855051963556
0 2 0.850455666909
0 3 0.838426165931
1 0 0.875429650128
1 1 0.891505756083
1 2 0.877707700492
1 3 0.876479043451
2 0 0.889969666359
2 1 0.899792354281
2 2 0.905744404886
2 3 0.89723012972
3 0 0.891406536654
3 1 0.929039774465
3 2 0.92639481212
3 3 0.933953585987


In [34]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    
    for aspect_j in aspects :
        aucs_1, aucs_2 = [], []
        diff = np.where(labs[aspect] != labs[aspect_j])[0]
        for i, j in enumerate(diff) :
            aucs_1.append(roc_auc_score(H[aspect_j][j], scores[j]))
            aucs_2.append(roc_auc_score(H[aspect][j], scores[j]))

        print aspect, aspect_j, np.mean(aucs_1), np.mean(aucs_2)

0 0 nan nan
0 1 0.370622286877 0.639396171127
0 2 0.365474339217 0.645146144333
0 3 0.352879303733 0.661331265471
1 0 0.28271214123 0.72691360183
1 1 nan nan
1 2 0.348389218007 0.655365019946
1 3 0.372001240086 0.632264282159
2 0 0.32830255246 0.681236980896
2 1 0.408681245031 0.593148601301
2 2 nan nan
2 3 0.422449328884 0.579977565423
3 0 0.145812273849 0.880433171665
3 1 0.204332165151 0.802590598688
3 2 0.23556738969 0.768500434829
3 3 nan nan


In [10]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.914870707499
0 1 0.883242027296
0 2 0.878992594595
0 3 0.86796036685
1 0 0.894897050529
1 1 0.929701394345
1 2 0.911345285882
1 3 0.9134474411
2 0 0.898631977632
2 1 0.920803103162
2 2 0.930926076498
2 3 0.926525263047
3 0 0.881059784896
3 1 0.934046852142
3 2 0.937719486865
3 3 0.952891638474
